# **Applied Data Science Capstone - Week 3**

# Part 1 - Get shape of data

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

In [21]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

In [22]:
table = soup.find('table', class_ = 'wikitable sortable') # Gets the table from the webpage
rows = table.find_all('tr') # Gets the table rows

postcodes = [] # Initializes the raw postcodes list
boroughs = [] # Initializes the raw boroughs list
neighbourhoods = [] # Initializes the raw neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        # Skip if the borough name is 'Not Assigned' - Removing the newline character at the en   

        if columns[1].text != 'Not assigned\n':
            
            postcode = columns[0].text.split('\n')[0]
            postcodes.append(postcode)
                                            
            borough = columns[1].text.split('\n')[0]
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0]      
            # Assigning the same name to neighbourhood if it is 'Not Assigned'
            if neighbourhood == 'Not assigned':
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
        
    except Exception as e : # Skip the first row which contains column names
        pass 

    
postcode_explored = [] # Initializing 
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)
        

In [23]:
csv_file.close()

In [24]:
df = pd.read_csv('toronto_postal_codes.csv')
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [26]:
df.shape

(103, 3)

## This is the end of Part 1, now comes with Part 2....

# Part2 - Get Latitude and Logitude

In [33]:
df_geocoder = pd.read_csv('Geospatial_Coordinates.csv') #geting geocode from mentioned csv
df_geocoder.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [80]:
latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array
postal_codes = df['Postcode']

In [81]:
#Mapping postal code between two data frames
for postal_code in postal_codes : 
    lat = df_geocoder[df_geocoder['Postal Code']== postal_code]['Latitude'].iloc[0]
    lon = df_geocoder[df_geocoder['Postal Code'] == postal_code]['Longitude'].iloc[0]
    
    latitudes.append(lat)
    longitudes.append(lon)

In [82]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [83]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## This is the end of Part 2, now comes with Part 3....

# Part3 - Explore and cluster the neighborhoods in Toronto

In [107]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library
import requests
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [164]:
CLIENT_ID = '*********' # your Foursquare ID
CLIENT_SECRET = '******' # your Foursquare Secret
ACCESS_TOKEN = '******' # your FourSquare Access Token {"access_token":"PG1GWWTYRRMWEGQZ3AU4NHZINGSYCY1RK3UAWUR4MW3ZKDVV"}
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *********
CLIENT_SECRET:******


**1.Display the map of Toronto**

In [88]:
# Toronto latitude and longitude using Google search
tor_lat = 43.6532
tor_lng = -79.3832

# Creates map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lng], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**2.get Venues of Toronto**

In [165]:
latitude = 43.6532
longitude = -79.3832
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=*********&client_secret=******&ll=43.6532,-79.3832&v=20180604&radius=500&limit=30'

In [109]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5227bb01498e17bf485e6202',
  'name': 'Downtown Toronto',
  'location': {'lat': 43.65323167517444,
   'lng': -79.38529600606677,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65323167517444,
     'lng': -79.38529600606677}],
   'distance': 168,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['Toronto ON', 'Canada']},
  'categories': [{'id': '4f2a25ac4b909258e854f55f',
    'name': 'Neighborhood',
    'pluralName': 'Neighborhoods',
    'shortName': 'Neighborhood',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-5227bb01498e17bf485e6202-0'}

In [110]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Downtown Toronto,Neighborhood,NaN,CA,Toronto,Canada,NaN,168,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65323167517444...",43.653232,-79.385296,NaN,NaN,ON,5227bb01498e17bf485e6202
1,Nathan Phillips Square,Plaza,100 Queen St W,CA,Toronto,Canada,at Bay St,106,"[100 Queen St W (at Bay St), Toronto ON M5H 2N...","[{'label': 'display', 'lat': 43.65227047322295...",43.652270,-79.383516,NaN,M5H 2N1,ON,4ad4c05ef964a520a6f620e3
2,Indigo,Bookstore,220 Yonge St,CA,Toronto,Canada,NaN,204,"[220 Yonge St, Toronto ON M5B 2H1, Canada]","[{'label': 'display', 'lat': 43.65351471121164...",43.653515,-79.380696,Downtown Yonge,M5B 2H1,ON,4b2a6eb8f964a52012a924e3
3,LUSH,Cosmetics Shop,"220 Yonge St, Unit B215-A",CA,Toronto,Canada,in Toronto Eaton Centre,228,"[220 Yonge St, Unit B215-A (in Toronto Eaton C...","[{'label': 'display', 'lat': 43.653557, 'lng':...",43.653557,-79.380400,NaN,M5B 2H1,ON,4bd0b30d41b9ef3b8fa0fae5
4,Old City Hall,Monument / Landmark,60 Queen Street West,CA,Toronto,Canada,NaN,177,"[60 Queen Street West, Toronto ON M5H 1A1, Can...","[{'label': 'display', 'lat': 43.65200880087612...",43.652009,-79.381744,NaN,M5H 1A1,ON,4ad4c05ef964a5208ef620e3
5,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,CA,Toronto,Canada,btwn Queen & Dundas,228,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",NaN,43.654447,-79.380952,"Downtown Toronto, Toronto, ON",M5B 2H1,ON,4ad77a12f964a520260b21e3
6,M Square Coffee Co,Coffee Shop,C24 - 123 Queen St W,CA,Toronto,Canada,NaN,222,"[C24 - 123 Queen St W, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65121797253777...",43.651218,-79.383555,NaN,NaN,ON,54132b3b498ee9ca9332e189
7,Japango,Sushi Restaurant,122 Elizabeth St.,CA,Toronto,Canada,at Dundas St. W,279,"[122 Elizabeth St. (at Dundas St. W), Toronto ...","[{'label': 'display', 'lat': 43.65526771691681...",43.655268,-79.385165,NaN,M5G 1P5,ON,4ae7b27df964a52068ad21e3
8,Crepe Delicious,Fast Food Restaurant,220 Yonge St.,CA,Toronto,Canada,NaN,238,"[220 Yonge St., Toronto ON M5B 2H1, Canada]","[{'label': 'display', 'lat': 43.65453648827724...",43.654536,-79.380889,NaN,M5B 2H1,ON,4e5d8181a8092f63968617ee
9,Poke Guys,Poke Place,112 Elizabeth St,CA,Toronto,Canada,at Dundas St W,240,"[112 Elizabeth St (at Dundas St W), Toronto ON...","[{'label': 'display', 'lat': 43.65489527525682...",43.654895,-79.385052,NaN,M5G 1P5,ON,57bcd3b7498e652a678d0378


In [111]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

**3. Get data of Downtown Toronto and start explore**

In [120]:
downtown_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [126]:
downtown_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

**4.Get geo info of 'Regent Park, Harbourfront'**

In [129]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [166]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=*********&client_secret=******&v=20180604&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [131]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600c0cdb4b6c4351f2839de4'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

**5. Explore Neighborhoods in Regent Park, Harbourfront** 

In [132]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**6.Run the above function on each neighborhood and create a new dataframe called tor_venues.**

In [134]:
tor_venues = getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [135]:
print(tor_venues.shape)
tor_venues.head()

(1225, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


**7.Check how many venues were returned for each neighborhood**

In [136]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [137]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 206 uniques categories.


**8.Analyze Each Neighborhood**

In [140]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [141]:
tor_onehot.shape

(1225, 206)

In [142]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,...,0.017241,0.000000,0.000000,0.00,0.00,0.017241,0.00000,0.00,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.0625,0.1875,0.125,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000
2,Central Bay Street,0.016393,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,...,0.016393,0.000000,0.000000,0.00,0.00,0.016393,0.00000,0.00,0.016393,0.000000
3,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000
4,Church and Wellesley,0.025641,0.012821,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.012821,...,0.012821,0.012821,0.012821,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.040000,...,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.00000,0.00,0.010000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.030000,...,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.00,0.010000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,...,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.01000,0.01,0.010000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,...,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.00,0.010000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.00,0.050000,0.00000,0.05,0.016667,0.000000


In [143]:
tor_grouped.shape

(19, 206)

**9. print each neighborhood along with the top 5 most common venues**

In [144]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3      Farmers Market  0.03
4  Seafood Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1  Airport Terminal  0.12
2             Plane  0.06
3   Harbor / Marina  0.06
4          Boutique  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.07
2      Sandwich Place  0.05
3  Italian Restaurant  0.05
4     Bubble Tea Shop  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3      Nightclub  0.06
4     Baby Store  0.06


----Church and Wellesley----
                  venue  freq
0           Coffee Shop  0.09
1   Japanese Restaurant  0.06
2      Sushi Restaurant  0.06
3  Fast Food Restaurant  0.04
4   

In [145]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [146]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Bakery,Shopping Mall,Eastern European Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Plane,Boutique,Rental Car Location,Sculpture Garden,Harbor / Marina,Airport Lounge,Airport Gate,Airport Food Court
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bubble Tea Shop,Burger Joint,Salad Place,Poke Place,Pizza Place,Middle Eastern Restaurant
3,Christie,Grocery Store,Café,Park,Coffee Shop,Baby Store,Restaurant,Candy Store,Italian Restaurant,Athletics & Sports,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Café,Pizza Place


**10.Cluster Neighborhoods**

In [147]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

In [155]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels3', kmeans.labels_)

tor_merged = downtown_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

tor_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,4,4,4,Coffee Shop,Pub,Park,Bakery,Restaurant,Theater,Breakfast Spot,Café,Electronics Store,Performing Arts Venue
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,4,4,4,Coffee Shop,Sushi Restaurant,Gym,Fast Food Restaurant,Restaurant,Portuguese Restaurant,Park,Nightclub,Mexican Restaurant,Japanese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0,0,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Bubble Tea Shop,Fast Food Restaurant,Lingerie Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0,0,0,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Gym,Clothing Store,Beer Bar,Hotel,Seafood Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,0,0,0,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Bakery,Shopping Mall,Eastern European Restaurant


In [157]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels3']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**11.Examine Clusters**

Cluster1

In [159]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,0,0,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Bubble Tea Shop,Fast Food Restaurant,Lingerie Store
3,Downtown Toronto,0,0,0,0,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Gym,Clothing Store,Beer Bar,Hotel,Seafood Restaurant
4,Downtown Toronto,0,0,0,0,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Bakery,Shopping Mall,Eastern European Restaurant
7,Downtown Toronto,0,0,0,0,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Thai Restaurant,Deli / Bodega,Sushi Restaurant,Pizza Place,Burrito Place
8,Downtown Toronto,0,0,0,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Brewery,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Park
9,Downtown Toronto,0,0,0,0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Salad Place,American Restaurant,Seafood Restaurant,Sporting Goods Shop
10,Downtown Toronto,0,0,0,0,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega
11,Downtown Toronto,0,0,0,0,Café,Bookstore,Bar,Japanese Restaurant,Bakery,Yoga Studio,Italian Restaurant,Beer Bar,College Gym,Sandwich Place
12,Downtown Toronto,0,0,0,0,Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Gaming Cafe,Farmers Market,Dessert Shop,Grocery Store,Caribbean Restaurant
15,Downtown Toronto,0,0,0,0,Coffee Shop,Seafood Restaurant,Beer Bar,Hotel,Restaurant,Cocktail Bar,Italian Restaurant,Café,Japanese Restaurant,Gym


cluster2

In [160]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,1,1,1,Park,Trail,Playground,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


Cluster3

In [161]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,2,2,2,2,Grocery Store,Café,Park,Coffee Shop,Baby Store,Restaurant,Candy Store,Italian Restaurant,Athletics & Sports,Nightclub


Cluser4

In [162]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,3,3,3,3,Airport Service,Airport Terminal,Plane,Boutique,Rental Car Location,Sculpture Garden,Harbor / Marina,Airport Lounge,Airport Gate,Airport Food Court


Cluster5

In [163]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,4,4,4,Coffee Shop,Pub,Park,Bakery,Restaurant,Theater,Breakfast Spot,Café,Electronics Store,Performing Arts Venue
1,Downtown Toronto,4,4,4,4,Coffee Shop,Sushi Restaurant,Gym,Fast Food Restaurant,Restaurant,Portuguese Restaurant,Park,Nightclub,Mexican Restaurant,Japanese Restaurant
5,Downtown Toronto,4,4,4,4,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bubble Tea Shop,Burger Joint,Salad Place,Poke Place,Pizza Place,Middle Eastern Restaurant


# Thank you for Reviewing!